In [1]:
import os
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import Phylo
import pandas as pd
import seaborn as sns
mpl.rcParams['pdf.fonttype'] = 42
%matplotlib inline

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def checkSlash(directory):
    if directory[-1] != '/':
        directory = directory + '/'
    return directory

In [3]:
coreDir = '/home/ubuntu/proc/sjspence/170105_PSE/'
snpDir = '11_parsnp_subgroups_snps'
subDir = 'subA/'
coreDir = checkSlash(coreDir)
snpDir = checkSlash(snpDir)
subDir = checkSlash(subDir)

**SETUP ANCESTRAL RECONSTRUCTION**

In [16]:
ctlName = 'baseml.ctl'
ctl = open(ctlName, 'r')
ctlOut = open(snpDir + subDir + ctlName, 'w')
#Files to access
variantFile = coreDir + snpDir + subDir + 'variants.mfa'
treeFile = coreDir + snpDir + subDir + 'parsnp.tree'
outFile = coreDir + snpDir + subDir + 'mlb'
for line in ctl:
    lineList = line.strip().split(' ')
    if lineList[0] == 'seqfile':
        ctlOut.write(line.replace(lineList[2], variantFile))
    elif lineList[0] == 'treefile':
        ctlOut.write(line.replace(lineList[2], treeFile))
    elif lineList[0] == 'outfile':
        ctlOut.write(line.replace(lineList[2], outFile))
    elif lineList[0] == 'model':
        ctlOut.write(line.replace(lineList[2], '0')) #Jukes-cantor 69
    elif lineList[0] == 'RateAncestor':
        ctlOut.write(line.replace(lineList[2], '1'))
#    elif lineList[0] == 'nhomo':
#        ctlOut.write('* ' + line)
    else:
        ctlOut.write(line)
ctl.close()
ctlOut.close()

In [17]:
os.system('baseml ' + snpDir + subDir + ctlName)

65280

**ANCESTRAL RECONSTRUCTION WITH FASTML**

In [4]:
#First, edit tree files to remove non-standard quote characters
treeFile = coreDir + snpDir + subDir + 'parsnp.tree'
treeIn = open(treeFile, 'r')
treeEdit = treeFile.replace('.tree', '_edit.tree')
treeOut = open(treeEdit, 'w')
for line in treeIn:
    treeOut.write(line.replace('\'', ''))
treeIn.close()
treeOut.close()

In [31]:
tree2name = treeEdit.replace('.tree', '_tree2.txt')
tree1 = open(treeEdit, 'r')
tree2 = open(tree2name, 'w')
for line in tree1:
    line = line.replace('_contigs.fa.ref', '-ref')
    line = line.replace('_contigs.fa', '')
    line = line.replace('-','_')
    line = line.replace('D17', '')
    line = line.replace('_ref', '')
    tree2.write(line)
#START HERE, RENAME BY IMPORTING TREES AND RENAMING NODES
tree1.close()
tree2.close()

3
(((0_102050:0.10163,_102044:0.12219)0.380:0.01083,((_102045:0.16780,_102046:0.12594)0.840:0.02093,(_102047:0.11969,(_102040:0.13137,_102043:0.14422)0.110:0.01392)0.890:0.01294)0.950:0.02018)0.940:0.01913,((_102037:0.11354,_102048:0.19374)0.450:0.02021,(_102042:0.14147,_102065:0.11813)0.770:0.02021)0.270:0.00468,((_102036:0.14163,_102051:0.10484)0.970:0.02329,((_102039:0.09703,_102038:0.15359)0.980:0.03284,(_102041:0.13162,_102049:0.12741)0.920:0.02011)0.810:0.00357)0.930:0.01292);

19
(((_102050:0.10163,1_102044:0.12219)0.380:0.01083,((_102045:0.16780,_102046:0.12594)0.840:0.02093,(_102047:0.11969,(_102040:0.13137,_102043:0.14422)0.110:0.01392)0.890:0.01294)0.950:0.02018)0.940:0.01913,((_102037:0.11354,_102048:0.19374)0.450:0.02021,(_102042:0.14147,_102065:0.11813)0.770:0.02021)0.270:0.00468,((_102036:0.14163,_102051:0.10484)0.970:0.02329,((_102039:0.09703,_102038:0.15359)0.980:0.03284,(_102041:0.13162,_102049:0.12741)0.920:0.02011)0.810:0.00357)0.930:0.01292);

51
(((_102050:0.10163

In [11]:
variantFileName = coreDir + snpDir + subDir + 'variants.mfa'
variantFile = open(variantFileName, 'r')
variant2name = variantFileName.replace('.mfa', '2.aln')
variant2 = open(variant2name, 'w')
for line in variantFile:
    if '>' in line:
        line = line.replace('_contigs.fa.ref', '-ref')
        line = line.replace('_contigs.fa', '')
        line = line.replace('-','_')
        line = line.replace('D17_', '')
        line = line.replace('_ref', '')
        variant2.write(line)
    else:
        variant2.write(line)
variantFile.close()
variant2.close()

In [12]:
fastmlPath = '/home/ubuntu/tools/FastML.v3.1/www/fastml/FastML_Wrapper.pl'
outDir = coreDir + snpDir + subDir + 'fastml/'
if not os.path.exists(outDir):
    os.makedirs(outDir)

In [13]:
outDir = coreDir + snpDir + subDir + 'fastml/'
os.system('rm -r ' + outDir)
os.system('perl ' + fastmlPath + ' --MSA_File ' + variant2name + ' --seqType nuc --outDir ' + outDir + \
         ' --Tree ' + tree2name + ' --jointReconstruction no --indelReconstruction ML')

0

In [9]:
print(variant2name)
print(tree2name)

/home/ubuntu/proc/sjspence/170105_PSE/11_parsnp_subgroups_snps/subA/variants2.aln
/home/ubuntu/proc/sjspence/170105_PSE/11_parsnp_subgroups_snps/subA/parsnp_edit_tree2.txt


In [37]:
#Demonstrate that this damn program *can* work
alnFile = coreDir + snpDir + subDir + 'seq.aln'
treeFile = coreDir + snpDir + subDir + 'tree.txt'
outDir = coreDir + snpDir + subDir + 'trial/'
os.system('perl ' + fastmlPath + ' --MSA_File ' + alnFile + ' --outDir ' + outDir + ' --seqType aa --Tree ' + \
          treeFile)

0

In [39]:
Phylo.draw_ascii(Phylo.read(treeFile, 'newick'))

  __________________________ 8_Q98862_Q
 |
 |     _____________________ 7_Q91612_Q
 |    |
 |    |                                        _ 0__reading
 |____|         ______________________________|
 |    |        |                              , 9_HH_DROME
 |    |        |                              |
 |    |        |                              | 10_A46400
_|    |________|
 |             |                                                  , 1_Q09632_Q
 |             |    ______________________________________________|
 |             |   |                                              | 2_Q94128_Q
 |             |___|
 |                 |               _ 5_Q91610_Q
 |                 |______________|
 |                                | 6_Q91611_Q
 |
 | ______________ 3_Q90385_Q
 ||
  |__________ 4_Q90419_Q



In [5]:
testTree = Phylo.read(snpDir + 'subA/parsnp.tree', 'newick')

In [6]:
Phylo.draw_ascii(testTree)

        _____________________ D17-102050_contigs.fa.ref
      _|
     | |_________________________ D17-102044_contigs.fa
     |
  ___|        ____________________________________ D17-102045_contigs.fa
 |   |    ___|
 |   |   |   |___________________________ D17-102046_contigs.fa
 |   |___|
 |       |   ________________________ D17-102047_contigs.fa
 |       |__|
 |          |   ___________________________ D17-102040_contigs.fa
 |          |__|
 |             |______________________________ D17-102043_contigs.fa
 |
 |     _______________________ D17-102037_contigs.fa
_| ___|
 ||   |________________________________________ D17-102048_contigs.fa
 ||
 ||    _____________________________ D17-102042_contigs.fa
 ||___|
 |    |________________________ D17-102065_contigs.fa
 |
 |       ______________________________ D17-102036_contigs.fa
 |  ____|
 | |    |______________________ D17-102051_contigs.fa
 | |
 |_|        ____________________ D17-102039_contigs.fa
   | ______|
   ||      |__________

In [65]:
def getSNPs(contigFile):
    records = SeqIO.parse(snpDir + subDir + 'variants.mfa', 'fasta')
    for r in records:
        if r.id == contigFile:
            return(r.seq)
    print('Error, wrong file referenced')

def compareTerminals(terminals):
    for subList in terminals:
        commonClade = []
        for i in subList:
            variants = getSNPs(i)
            commonClade.append(variants)
        

In [66]:
def compareSNPs(clade):
    if clade.is_terminal():
        pass
    elif len(clade.clades) >= 2:
        terminals = []
        for c in clade.clades:
            terminalObjects = c.get_terminals()
            terminalList = []
            for o in terminalObjects:
                terminalList.append(o.name)
            terminals.append(terminalList)
        print(terminals)
        compareTerminals(terminals)
        for c in clade.clades:
            compareSNPs(c)
    else:
        print('Error, check tree configuration')

In [67]:
compareSNPs(testTree.root)

[['D17-102050_contigs.fa.ref', 'D17-102044_contigs.fa', 'D17-102045_contigs.fa', 'D17-102046_contigs.fa', 'D17-102047_contigs.fa', 'D17-102040_contigs.fa', 'D17-102043_contigs.fa'], ['D17-102037_contigs.fa', 'D17-102048_contigs.fa', 'D17-102042_contigs.fa', 'D17-102065_contigs.fa'], ['D17-102036_contigs.fa', 'D17-102051_contigs.fa', 'D17-102039_contigs.fa', 'D17-102038_contigs.fa', 'D17-102041_contigs.fa', 'D17-102049_contigs.fa']]
748
748
748
748
748
748
748
748
748
748
748
748
748
748
748
748
748
[['D17-102050_contigs.fa.ref', 'D17-102044_contigs.fa'], ['D17-102045_contigs.fa', 'D17-102046_contigs.fa', 'D17-102047_contigs.fa', 'D17-102040_contigs.fa', 'D17-102043_contigs.fa']]
748
748
748
748
748
748
748
[['D17-102050_contigs.fa.ref'], ['D17-102044_contigs.fa']]
748
748
[['D17-102045_contigs.fa', 'D17-102046_contigs.fa'], ['D17-102047_contigs.fa', 'D17-102040_contigs.fa', 'D17-102043_contigs.fa']]
748
748
748
748
748
[['D17-102045_contigs.fa'], ['D17-102046_contigs.fa']]
748
748
[['D

In [33]:
#[method for method in dir(testTree) if callable(getattr(testTree, method))]
#globals(testTree)
[method for method in dir(testTree.root) if callable(getattr(testTree.root, method))]

['__bool__',
 '__class__',
 '__delattr__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_filter_search',
 '_get_color',
 '_set_color',
 'collapse',
 'collapse_all',
 'common_ancestor',
 'count_terminals',
 'depths',
 'distance',
 'find_any',
 'find_clades',
 'find_elements',
 'get_nonterminals',
 'get_path',
 'get_terminals',
 'is_bifurcating',
 'is_monophyletic',
 'is_parent_of',
 'is_preterminal',
 'is_terminal',
 'ladderize',
 'prune',
 'split',
 'total_branch_length',
 'trace']

In [ ]:
def pop_list(nodes=None, parent=None, node_list=None):
    if parent is None:
        return node_list
    node_list.append([])
    for node in nodes:
        if node['parent'] == parent:
            node_list[-1].append(node)
        if node['id'] == parent:
            next_parent = node['parent']

    pop_list(nodes, next_parent, node_list)
    return node_list

In [ ]:
for clade in testTree.find_clades():
    print clade.name